In [10]:
import cv2
import sys, os
import numpy as np
import torch
from matplotlib import pyplot as plt
import pandas as pd
from collections import defaultdict

from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
#from segment_anything import SamPredictor, sam_model_registry
sys.path.append("/home/knuvi/Desktop/song/cucumber-image/")
from custom_utils import ensure_directories_exist

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print(f"using device: {device}")

if device.type == "cuda":
    # use bfloat16 for the entire notebook
    torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
    # turn on tfloat32 for Ampere GPUs (https://pytorch.org/docs/stable/notes/cuda.html#tensorfloat-32-tf32-on-ampere-devices)
    if torch.cuda.get_device_properties(0).major >= 8:
        torch.backends.cuda.matmul.allow_tf32 = True
        torch.backends.cudnn.allow_tf32 = True

using device: cuda


In [4]:
# 이미지와 레이블 경로
image_folder = '/home/knuvi/Desktop/song/cucumber-image/data/oi/images'
label_folder = '/home/knuvi/Desktop/song/cucumber-image/yolo_detection/oi_leaves2/labels'
mask_output_folder = '/home/knuvi/Desktop/song/cucumber-image/data/oi_leaves/mask'  # 마스크를 저장할 폴더 경로

sam2_checkpoint = '/home/knuvi/Desktop/song/cucumber-image/segment-anything-2/sam2_hiera_large.pt'
model_cfg = "sam2_hiera_l.yaml"

In [5]:
sam2 = build_sam2(model_cfg, sam2_checkpoint, device=device)
mask_predictor = SAM2ImagePredictor(sam2)


In [7]:
def save_images_with_new_name(bboxes, image_path, image_index, new_image_dir):
    # 새로운 이미지 이름 생성 (레이블 수를 포함)
    label_count = len(bboxes)
    new_image_name = f"oi_{str(image_index).zfill(3)}_{label_count}.jpg"

    # 이미지 저장 경로
    new_image_path = os.path.join(new_image_dir, new_image_name)
    
    # 이미지를 새 경로에 저장
    image = cv2.imread(image_path)
    if image is not None:
        cv2.imwrite(new_image_path, image)
        print(f"이미지 저장됨: {new_image_path}")
        return new_image_name
    else:
        print(f"이미지를 로드할 수 없음: {image_path}")
        return None

def load_bboxes_from_txt(txt_file):
    bboxes = []
    class_labels = []  # 각 bbox의 클래스 정보 저장
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        for line in lines:
            # txt 파일 형식: class x_min y_min x_max y_max
            class_id, x_min, y_min, x_max, y_max, _ = map(float, line.strip().split())
            if class_id == 0 or class_id == 1:  # class 0 (cucumber) 또는 class 1 (leaf)
                # SAM에 들어갈 bbox 형식: [x_min, y_min, x_max, y_max]
                bboxes.append([int(x_min), int(y_min), int(x_max), int(y_max)])
                class_labels.append(int(class_id))  # 클래스 정보를 저장
    return bboxes, class_labels
 
# 바운딩 박스를 저장하는 함수 (마스크 파일명과 동일하게 저장)
def save_bboxes_with_mask_name(bboxes, class_labels, mask_filename, label_dir):
    # 확장자 제거 (".png" 또는 다른 확장자를 제거)
    base_name = os.path.splitext(mask_filename)[0]
    label_filename = f"{base_name}.txt"
    label_full_path = os.path.join(label_dir, label_filename)
    
    # 바운딩 박스 정보를 텍스트 파일로 저장
    with open(label_full_path, 'w') as f:
        for bbox, class_id in zip(bboxes, class_labels):
            x_min, y_min, x_max, y_max = bbox
            # 저장 형식: class_id x_min y_min x_max y_max
            f.write(f"{class_id} {x_min} {y_min} {x_max} {y_max}\n")
    
    print(f"바운딩 박스 저장됨: {label_full_path}")

def generate_masks(image_path, bboxes, class_labels, score_threshold = 0.7):
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    masks_list = []
    filtered_class_labels = []  # 조건을 만족하는 클래스 레이블 저장
    filtered_bboxes = []  # 조건을 만족하는 bbox 저장
    
    for i, bbox in enumerate(bboxes):
        x_min, y_min, x_max, y_max = bbox
        box = np.array([x_min, y_min, x_max, y_max])  # bbox 좌표 설정
        
        # bbox의 중앙 좌표 계산
        center_x = (x_min + x_max) // 2
        center_y = (y_min + y_max) // 2
        input_point = np.array([[center_x, center_y]])  # bbox 중앙 좌표
        input_label = np.array([class_labels[i]])  # class number (0: cucumber, 1: leaf)

        # SAM 모델을 통한 마스크 예측
        mask_predictor.set_image(image_rgb)
        masks, scores, logits = mask_predictor.predict(
            box=box,
            multimask_output=True,
            #point_coords=input_point,
            point_labels=input_label
        )

        # 스코어가 가장 높은 마스크만 선택
        highest_score_idx = np.argmax(scores)
        highest_score = scores[highest_score_idx]
        best_mask = masks[highest_score_idx]  # 가장 높은 스코어의 마스크 선택
        if highest_score >= score_threshold:
            masks_list.append(best_mask)  # 조건을 만족하는 마스크 저장
            filtered_class_labels.append(class_labels[i])  # 해당 마스크의 클래스 레이블 저장
            filtered_bboxes.append(bbox)
            print(f"bbox {i + 1}: 가장 높은 score: {highest_score}로 저장됨")
        else:
            print(f"bbox {i + 1}: score {highest_score}로 저장되지 않음")
    
    return masks_list, filtered_class_labels, filtered_bboxes

In [8]:
def save_masks_with_new_name(masks, class_labels, bboxes, new_image_name, mask_dir, label_dir):
    # 확장자 제거 (".jpg" 또는 다른 확장자를 제거)
    base_name = os.path.splitext(new_image_name)[0]
    
    # 각 클래스 ID에 대해 파일명이 중복되지 않도록 번호 추가
    mask_counter = {}

    for idx, (mask, class_id, bbox) in enumerate(zip(masks, class_labels, bboxes)):
        # 해당 class_id에 대한 마스크 번호를 저장하고, 없으면 0으로 초기화
        if class_id not in mask_counter:
            mask_counter[class_id] = 0
        else:
            mask_counter[class_id] += 1

        # 마스크 파일명 생성 (중복 방지용 번호 포함)
        mask_number = str(mask_counter[class_id]).zfill(2)  # 두 자리 숫자로 포맷
        mask_filename = f"{base_name}_{class_id}_{mask_number}.png"
        mask_full_path = os.path.join(mask_dir, mask_filename)

        # 마스크 저장 (0과 255 값으로 이진화하여 저장)
        cv2.imwrite(mask_full_path, mask * 255)  # 이진화된 마스크를 PNG로 저장
        print(f"마스크 저장됨: {mask_full_path}")

        # 바운딩 박스 정보도 마스크 파일명과 동일하게 저장
        save_bboxes_with_mask_name([bbox], [class_id], mask_filename, label_dir)

def process_images_and_masks(image_paths, label_dir, new_image_path, mask_path, bbox_dir, start_index=1, score_threshold=0.5):
    image_index = start_index
    # 3개의 이미지에 대한 SAM 마스크 생성 및 시각화
    for image_path in image_paths:
        
        
        # 저장된 bbox 불러오기
        image_filename = os.path.basename(image_path)  # 이미지 파일 이름 추출
        txt_file = os.path.join(label_dir, os.path.splitext(image_filename)[0] + '.txt')  # 동일한 경로의 txt 파일
        if os.path.exists(txt_file):
            bboxes, class_labels = load_bboxes_from_txt(txt_file)
            if len(bboxes) == 0:
                print(f"레이블이 없는 이미지: {image_filename}")
                continue
        
        # 새로운 이름으로 이미지 저장
        new_image_name = save_images_with_new_name(bboxes, image_path, image_index, new_image_path)
        if new_image_name is None:
            continue

        # 유효한 bbox에 대해 SAM 마스크 생성
        if bboxes:
            masks, filtered_class_labels, filtered_bboxes = generate_masks(image_path, bboxes, class_labels, score_threshold=0.5)

            # 마스크, bbox 저장
            save_masks_with_new_name(masks, filtered_class_labels, filtered_bboxes, new_image_name, mask_path, bbox_dir)   
        else:
            print(f"No valid bboxes found for image {image_path}")

        image_index += 1

In [11]:
image_paths = [
    #'/home/knuvi/Desktop/song/cucumber-image/data/oi/images/V003_3_3_1_2_4_2_2_1_0_0_20221015_5068_20240422195045.jpg',
    '/home/knuvi/Desktop/song/cucumber-image/data/oi/images/V003_3_3_1_2_4_2_2_1_0_0_20221019_5096_20240422195046.jpg'
]

# bbox 정보가 저장된 경로 (YOLO 레이블이 있는 디렉토리)
label_dir = '/home/knuvi/Desktop/song/cucumber-image/data/whole_oi/inferenced_labels'
image_dir = '/home/knuvi/Desktop/song/cucumber-image/data/whole_oi/images'
new_image_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/images'
mask_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks'
bbox_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels'
ensure_directories_exist([new_image_dir, mask_dir, bbox_dir])

image_paths = []
for image_file in os.listdir(image_dir):
    if image_file.endswith(('.jpg', '.jpeg', '.png')):  # 이미지 파일만 처리
        image_path = os.path.join(image_dir, image_file)
        image_paths.append(image_path)

process_images_and_masks(image_paths, label_dir, new_image_dir, mask_dir, bbox_dir, start_index= 1, score_threshold=0.8)


디렉터리 생성됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/images
디렉터리 생성됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks
디렉터리 생성됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels
이미지 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/images/oi_001_1.jpg
bbox 1: 가장 높은 score: 0.9921875로 저장됨
마스크 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks/oi_001_1_1_00.png
바운딩 박스 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_001_1_1_00.txt
이미지 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/images/oi_002_1.jpg
bbox 1: 가장 높은 score: 0.99609375로 저장됨
마스크 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks/oi_002_1_1_00.png
바운딩 박스 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_002_1_1_00.txt
이미지 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/images/oi_003_1.jpg
bbox 1: 가장 높은 score: 0.99609375로 저장됨
마스크 저장됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks/oi_003_1_1_00.png
바운딩 박스 저

#### post processing

In [12]:
import os
import cv2
import numpy as np


# 마스크 폴더 경로
mask_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks'
label_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels'  # 바운딩 박스 정보가 저장된 폴더

# 마스크 및 바운딩 박스 파일 정리 함수
def clean_masks_and_bboxes_by_size(mask_list, area_threshold=0.30):
    total_deleted = 0  # 삭제된 마스크 개수 저장 변수
    # 마스크 파일들을 처리
    for mask_file in mask_list:
        # 마스크 파일 로드 (이진 마스크: 0과 255로만 구성)
        mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"마스크 파일을 불러오지 못함: {mask_file}")
            continue
        
        # 활성화된 마스크 픽셀 수 계산 (255로 된 픽셀만)
        mask_area = np.sum(mask == 255)
        
        # 마스크 이미지의 전체 픽셀 수
        total_pixels = mask.shape[0] * mask.shape[1]
        
        # 마스크가 이미지의 몇 %를 차지하는지 계산
        mask_area_ratio = mask_area / total_pixels
        
        # 마스크 파일명에서 확장자를 제거하고 바운딩 박스 파일명 생성
        base_name = os.path.splitext(os.path.basename(mask_file))[0]
        label_file = os.path.join(label_dir, f"{base_name}.txt")
        
        # 조건에 맞지 않는 마스크 파일 및 해당하는 바운딩 박스 파일 삭제
        if mask_area_ratio > area_threshold:
            os.remove(mask_file)
            if os.path.exists(label_file):
                os.remove(label_file)  # 바운딩 박스 파일도 삭제
                print(f"삭제됨: {label_file} (바운딩 박스)")
            total_deleted += 1  # 삭제된 마스크 개수 증가
            print(f"삭제됨: {mask_file} (차지 비율: {mask_area_ratio:.2f})")
        # else:
        #     print(f"유지됨: {mask_file} (차지 비율: {mask_area_ratio:.2f})")

    print(f"총 삭제된 마스크 및 바운딩 박스 개수: {total_deleted}")

# 마스크 경로 목록 생성
mask_paths = []
for mask_file in os.listdir(mask_dir):
    if mask_file.endswith(('.jpg', '.jpeg', '.png')):  # 이미지 파일만 처리
        mask_path = os.path.join(mask_dir, mask_file)
        mask_paths.append(mask_path)

# 마스크 파일 크기 조건 (전체 이미지의 30% 이하만 유지)
area_threshold = 0.3

# 마스크 및 바운딩 박스 정리 실행
clean_masks_and_bboxes_by_size(mask_paths, area_threshold=area_threshold)


삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_210_2_1_00.txt (바운딩 박스)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks/oi_210_2_1_00.png (차지 비율: 0.70)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_138_2_1_00.txt (바운딩 박스)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks/oi_138_2_1_00.png (차지 비율: 0.53)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_029_3_1_01.txt (바운딩 박스)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks/oi_029_3_1_01.png (차지 비율: 0.31)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_008_3_1_00.txt (바운딩 박스)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks/oi_008_3_1_00.png (차지 비율: 0.34)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_112_2_1_00.txt (바운딩 박스)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks/oi_112_2_1_00.png (차지 비율: 0.33)
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_188

In [ ]:
import os
import cv2
import numpy as np

# 경계선에 붙은 마스크를 확인하는 함수
def is_mask_touching_boundary(mask, boundary_margin=5):
    """
    마스크가 이미지 경계에 붙어 있는지 확인.
    
    Args:
        mask (numpy.ndarray): 이진 마스크 (0과 255로만 구성).
        boundary_margin (int): 경계에서 몇 픽셀까지를 경계로 간주할지.
    
    Returns:
        bool: 마스크가 경계에 붙어 있으면 True, 아니면 False.
    """
    h, w = mask.shape
    # 위쪽, 아래쪽, 왼쪽, 오른쪽 경계 영역 확인
    top_edge = mask[:boundary_margin, :]
    bottom_edge = mask[-boundary_margin:, :]
    left_edge = mask[:, :boundary_margin]
    right_edge = mask[:, -boundary_margin:]

    # 경계에 255(활성화된 픽셀)가 있으면 True
    if np.any(top_edge == 255) or np.any(bottom_edge == 255) or np.any(left_edge == 255) or np.any(right_edge == 255):
        return True
    return False

# 마스크 및 바운딩 박스 파일 정리 함수
def clean_masks_and_bboxes(mask_list, area_threshold=0.30, boundary_margin=5):
    total_deleted = 0  # 삭제된 마스크 개수 저장 변수
    for mask_file in mask_list:
        mask = cv2.imread(mask_file, cv2.IMREAD_GRAYSCALE)
        if mask is None:
            print(f"마스크 파일을 불러오지 못함: {mask_file}")
            continue
        
        # 마스크 활성화된 픽셀 비율 계산
        mask_area = np.sum(mask == 255)
        total_pixels = mask.shape[0] * mask.shape[1]
        mask_area_ratio = mask_area / total_pixels
        
        # 경계선 확인
        is_touching_boundary = is_mask_touching_boundary(mask, boundary_margin=boundary_margin)
        
        # 파일명 처리
        base_name = os.path.splitext(os.path.basename(mask_file))[0]
        label_file = os.path.join(label_dir, f"{base_name}.txt")
        
        # 조건에 맞지 않는 마스크 삭제
        if mask_area_ratio > area_threshold or is_touching_boundary:
            os.remove(mask_file)
            if os.path.exists(label_file):
                os.remove(label_file)
                print(f"삭제됨: {label_file} (바운딩 박스)")
            total_deleted += 1
            print(f"삭제됨: {mask_file} (비율: {mask_area_ratio:.2f}, 경계 접촉: {is_touching_boundary})")
    
    print(f"총 삭제된 마스크 및 바운딩 박스 개수: {total_deleted}")

# 경로 목록 생성 및 정리 실행
mask_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks'
label_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels'
mask_paths = [os.path.join(mask_dir, mask_file) for mask_file in os.listdir(mask_dir) if mask_file.endswith(('.jpg', '.jpeg', '.png'))]

clean_masks_and_bboxes(mask_paths, area_threshold=0.30, boundary_margin=5)


In [12]:
import os

# 폴더 경로
mask_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/masks'
label_dir = '/home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels'

# 마스크 폴더에서 현재 남아있는 파일 리스트 가져오기
remaining_mask_files = {os.path.splitext(mask_file)[0] for mask_file in os.listdir(mask_dir) if mask_file.endswith(('.jpg', '.jpeg', '.png'))}

# 레이블 폴더에서 제거할 파일 탐지 및 삭제
total_deleted = 0
for label_file in os.listdir(label_dir):
    if label_file.endswith('.txt'):
        base_name = os.path.splitext(label_file)[0]
        if base_name not in remaining_mask_files:  # 마스크 폴더에 없는 파일과 동일한 레이블 삭제
            label_path = os.path.join(label_dir, label_file)
            os.remove(label_path)
            total_deleted += 1
            print(f"삭제됨: {label_path}")

print(f"총 삭제된 레이블 파일 수: {total_deleted}")


삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_514_4_1_00.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_586_5_1_03.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_697_5_1_00.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_483_4_1_01.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_692_3_1_01.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_421_2_1_00.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_447_3_1_00.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_223_5_1_03.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_125_3_1_01.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_539_3_1_00.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_586_5_1_01.txt
삭제됨: /home/knuvi/Desktop/song/cucumber-image/data/oi_seg/labels/oi_597_3_1_01.txt
삭제됨: /home/knuvi